Zadanie 1: Pobieranie i analiza terminów leczenia z API NFZ

Cel zadania 
1. Pobranie danych z API NFZ z endpointu /queues dla 3 województw. 
2. Przetworzenie danych z formatu JSON do tabeli przy użyciu pandas i funkcji json_normalize. 
3. Wczytanie danych do lokalnej bazy SQLite. 
4. Przygotowanie podsumowania zebranych informacji przy użyciu Pythona. 

Biblioteki

In [6]:
import requests
import pandas as pd
import sqlite3
import time

1. Pobieranie danych

Zdecydowano się na 3 województwa:
- Dolnośląskie (01)
- Kujawsko-Pomorskie (02)
- Pomorskie (11)

Wykorzystujemy zasób API: GET /queues

Opis w dokumentacji:  "Zasób zwraca listę pierwszych dostępnych terminów leczenia".

Parametry:
- province: Kod województwa.
- case=1: Przypadki stabilne.
- limit=25: Pobieramy maksymalną dozwoloną liczbę wyników na stronę, aby przyspieszyć proces.

Struktura odpowiedzi: Dane znajdują się w sekcji data , a szczegóły w attributes
Pętla pobiera dane strona po stronie, aż do wyczerpania wyników. Ustawiono limit=25 (maksimum wg dokumentacji ), aby zminimalizować liczbę zapytań

In [8]:
wojewodztwa = {'01': 'Dolnośląskie', '02': 'Kuj-Pom', '11': 'Pomorskie'}
wszystkie_dane = []
domena = "https://api.nfz.gov.pl"

print("Rozpoczęcie pobierania danych...")

for kod, nazwa in wojewodztwa.items():
    print(f"--> Analiza województwa: {nazwa}")
    
    # Adres startowy
    url = f"{domena}/app-itl-api/queues?format=json&province={kod}&case=1&limit=25"
    
    licznik_stron = 1
    
    while url:
        try:
            odpowiedz = requests.get(url)
            if odpowiedz.status_code != 200:
                print(f"Błąd połączenia: {odpowiedz.status_code}")
                break
            
            dane_json = odpowiedz.json()
            
            # Wyświetlenie ile łącznie jest rekordów (tylko przy 1. stronie) ---
            if licznik_stron == 1:
                # Pobranie liczby z metadanych 
                ile_razem = dane_json.get('meta', {}).get('count', 'nieznana ilość')
                print(f"    Znaleziono {ile_razem} terminów. Pobieranie może potrwać...")

            # Pobranie danych
            lista_wynikow = dane_json.get('data', [])
            for wynik in lista_wynikow:
                if 'attributes' in wynik:
                    rekord = wynik['attributes']
                    rekord['Wojewodztwo'] = nazwa
                    wszystkie_dane.append(rekord)
            
            # Obsługa następnej strony
            if 'links' in dane_json and 'next' in dane_json['links']:
                nastepny_link = dane_json['links']['next']
                if nastepny_link and not nastepny_link.startswith('http'):
                    url = domena + nastepny_link
                else:
                    url = nastepny_link
            else:
                url = None

            # Informacja o postępie co 10 stron ---
            if licznik_stron % 10 == 0:
                print(f"    Pobrano stronę nr {licznik_stron}...")
            
            licznik_stron += 1
            time.sleep(0.05)
            
        except Exception as e:
            print(f"Błąd: {e}")
            break

print(f"\nZakończono! Łącznie pobrano {len(wszystkie_dane)} rekordów.")

Rozpoczęcie pobierania danych...
--> Analiza województwa: Dolnośląskie
    Znaleziono 5596 terminów. Pobieranie może potrwać...
    Pobrano stronę nr 10...
    Pobrano stronę nr 20...
    Pobrano stronę nr 30...
    Pobrano stronę nr 40...
    Pobrano stronę nr 50...
    Pobrano stronę nr 60...
    Pobrano stronę nr 70...
    Pobrano stronę nr 80...
    Pobrano stronę nr 90...
    Pobrano stronę nr 100...
    Pobrano stronę nr 110...
    Pobrano stronę nr 120...
    Pobrano stronę nr 130...
    Pobrano stronę nr 140...
    Pobrano stronę nr 150...
    Pobrano stronę nr 160...
    Pobrano stronę nr 170...
    Pobrano stronę nr 180...
    Pobrano stronę nr 190...
    Pobrano stronę nr 200...
    Pobrano stronę nr 210...
    Pobrano stronę nr 220...
--> Analiza województwa: Kuj-Pom
    Znaleziono 4025 terminów. Pobieranie może potrwać...
    Pobrano stronę nr 10...
    Pobrano stronę nr 20...
    Pobrano stronę nr 30...
    Pobrano stronę nr 40...
    Pobrano stronę nr 50...
    Pobrano s

2. Przetworzenie danych

In [ ]:
# Przekształcenie listy danych JSON na tabelę
df = pd.json_normalize(wszystkie_dane)

# Zdefiniowanie listy kolumn wymaganych w zadaniu
kolumny_do_wybrania = [
    'benefit',                                 # Nazwa świadczenia
    'provider',                                # Nazwa świadczeniodawcy
    'place',                                   # Miejsce udzielania świadczeń
    'address',                                 # Adres
    'locality',                                # Miejscowość
    'statistics.provider-data.awaiting',       # Liczba oczekujących
    'statistics.provider-data.average-period', # Średni czas oczekiwania
    'dates.date',                              # Data pierwszego wolnego terminu
    'Wojewodztwo'
]

# Wyselekcjonowanie tylko wymaganych kolumn
# Używamy .copy(), aby stworzyć niezależną kopię tabeli
df = df[kolumny_do_wybrania].copy()

# Zmiana nazw kolumn na prostsze
df.columns = [
    'Swiadczenie', 
    'Dostawca', 
    'Miejsce', 
    'Adres', 
    'Miejscowosc', 
    'Liczba_Oczekujacych', 
    'Sredni_Czas', 
    'Data_Terminu', 
    'Wojewodztwo'
]

# Konwersja danych liczbowych
# fillna(0) wstawia 0 tam, gdzie brakuje danych
# astype(int) zamienia liczby z przecinkiem na całkowite
df['Liczba_Oczekujacych'] = pd.to_numeric(df['Liczba_Oczekujacych']).fillna(0).astype(int)
df['Sredni_Czas'] = pd.to_numeric(df['Sredni_Czas']).fillna(0).astype(int)

print(f"Utworzono tabelę o wymiarach: {df.shape}")
df.head()

Utworzono tabelę o wymiarach: (13232, 9)


,Swiadczenie,Dostawca,Miejsce,Adres,Miejscowosc,Liczba_Oczekujacych,Sredni_Czas,Data_Terminu,Wojewodztwo
0,ZABIEGI W ZAKRESIE GRUCZOŁU KROKOWEGO (PROSTAT...,SPECJALISTYCZNY SZPITAL IM. DRA ALFREDA SOKOŁO...,ODDZIAŁ UROLOGICZNY,UL. SOKOŁOWSKIEGO 4,WAŁBRZYCH,0,0,2025-12-21,Dolnośląskie
1,PROFILAKTYKA ZAKAŻEŃ WIRUSEM RS,WOJEWÓDZKI SZPITAL SPECJALISTYCZNY WE WROCŁAWIU,ODDZIAŁ KARDIOLOGII DZIECIĘCEJ Z PODODDZIAŁEM ...,UL. HENRYKA MICHAŁA KAMIEŃSKIEGO 73A,WROCŁAW-PSIE POLE,0,0,2025-12-22,Dolnośląskie
2,ZABIEGI W ZAKRESIE TERMOLEZJI I BLOKADY,WOJEWÓDZKI SZPITAL SPECJALISTYCZNY WE WROCŁAWIU,ODDZIAŁ CHIRURGII ONKOLOGICZNEJ,UL. HENRYKA MICHAŁA KAMIEŃSKIEGO 73A,WROCŁAW-PSIE POLE,0,0,2025-12-22,Dolnośląskie
3,STACJA DIALIZ,UNIWERSYTECKI SZPITAL KLINICZNY IM. JANA MIKUL...,STACJA DIALIZ,UL. BOROWSKA 213,WROCŁAW,0,0,2025-12-22,Dolnośląskie
4,PORADNIA STOMATOLOGICZNA,PRYWATNA PRZYCHODNIA LEKARSKA STEFAN SKROCKI S...,PORADNIA STOMATOLOGICZNA,UL. UL. GRUNWALDZKA 65,WROCŁAW,0,0,2025-12-22,Dolnośląskie


3. Zapis do lokalnej bazy SQLite

In [10]:
# Nawiązanie połączenia z bazą danych (plik zostanie utworzony automatycznie)
polaczenie = sqlite3.connect('nfz_data.db')

# Zapisanie tabeli 'df' do bazy danych pod nazwą 'nfz_queues'
# Parametr if_exists='replace' ---> oznacza, że jeśli tabela już istnieje, zostanie nadpisana
df.to_sql('nfz_queues', polaczenie, if_exists='replace', index=False)

# Zamknięcie połączenia z bazą
polaczenie.close()

print("Zapisanie danych do pliku nfz_data.db zakończone powodzeniem.")

Zapisanie danych do pliku nfz_data.db zakończone powodzeniem.


4. Podsumowanie danych

In [11]:
print("Podsumowanie danych")

# 1. Wyświetlenie całkowitej liczby rekordów w tabeli
print(f"1. Całkowita liczba rekordów: {len(df)}")

# 2. Obliczenie łącznej liczby oczekujących w poszczególnych województwach
print("\n2. Liczba oczekujących wg województw:")
# Grupowanie po kolumnie 'Wojewodztwo' i sumowanie kolumny 'Liczba_Oczekujacych'
print(df.groupby('Wojewodztwo')['Liczba_Oczekujacych'].sum())

# 3. Obliczenie średniego czasu oczekiwania na świadczenia
srednia_dni = df['Sredni_Czas'].mean()
print(f"\n3. Średni czas oczekiwania (ogółem): {srednia_dni:.0f} dni")

# 4. Wyświetlenie TOP 5 miejsc z największą liczbą oczekujących
# Sortowanie malejące (od największej) i pobranie 5 pierwszych wierszy
top_5 = df.sort_values(by='Liczba_Oczekujacych', ascending=False).head(5)

print("\n4. TOP 5 miejsc świadczeń z największą kolejką:")
# Wyświetlenie tylko wybranych, czytelnych kolumn
print(top_5[['Dostawca', 'Miejscowosc', 'Wojewodztwo', 'Liczba_Oczekujacych']].to_string(index=False))

Podsumowanie danych
1. Całkowita liczba rekordów: 13232

2. Liczba oczekujących wg województw:
Wojewodztwo
Dolnośląskie    461397
Kuj-Pom         407325
Pomorskie       521160
Name: Liczba_Oczekujacych, dtype: int64

3. Średni czas oczekiwania (ogółem): 80 dni

4. TOP 5 miejsc świadczeń z największą kolejką:
                                                                                     Dostawca  Miejscowosc  Wojewodztwo  Liczba_Oczekujacych
SP ZOZ SZPITAL SPECJALISTYCZNY MINISTERSTWA SPRAW WNĘTRZNYCH I ADMINISTRACJI W JELENIEJ GÓRZE JELENIA GÓRA Dolnośląskie                 6143
                                  OŚRODEK PROFILAKTYKI I ROZWIĄZYWANIA PROBLEMÓW ALKOHOLOWYCH    WEJHEROWO    Pomorskie                 5445
                                            SPZOZ 10 WOJSKOWY SZPITAL KLINICZNY Z POLIKLINIKĄ    BYDGOSZCZ      Kuj-Pom                 4206
                                                             UNIWERSYTECKIE CENTRUM KLINICZNE       GDAŃSK    Pomorskie       